In [1]:
import pandas as pd
import re
import numpy as np
from collections import Counter

In [2]:
df = pd.read_csv("puzzle13.txt", header=None, sep=" ")
df.columns = ["hand", "bid"]

In [3]:
set("AAAB")

{'A', 'B'}

In [4]:

c = Counter("QQQJA")
len(c)
sorted(c)

['A', 'J', 'Q']

In [5]:
"""
6: Five of a kind, where all five cards have the same label: AAAAA
5: Four of a kind, where four cards have the same label and one card has a different label: AA8AA
4: Full house, where three cards have the same label, and the remaining two cards share a different label: 23332
3: Three of a kind, where three cards have the same label, and the remaining two cards are each different from any other card in the hand: TTT98
2: Two pair, where two cards share one label, two other cards share a second label, and the remaining card has a third label: 23432
1: One pair, where two cards share one label, and the other three cards have a different label from the pair and each other: A23A4
0: High card, where all cards' labels are distinct: 23456
"""

def calculate_strength(hand):
    counter = Counter(hand)
    # high card
    if len(counter) == 5:
        return 0
    if len(counter) == 4:
        return 1
    if len(counter) == 3:
        if counter.most_common()[1][1] == 2:
            return 2
        return 3
    if len(counter) == 2:
        if counter.most_common()[0][1] == 4:
            return 5
        return 4
    if len(counter) == 1:
        return 6
    
def calculate_strength_joker(hand):
    if "J" in hand:
        chars = ["A", "K", "Q", "T", "9", "8", "7", "6", "5", "4", "3", "2"]
        import re
        subs = []
        for char in chars:
            new_hand = re.sub('J', char, hand, 1)
            subs.append(calculate_strength_joker(new_hand))
        return max(subs)
    return calculate_strength(hand)

In [6]:
calculate_strength_joker("2222J")

6

In [7]:
calculate_strength("2222Q")

5

In [8]:
counter = Counter("2222Q")
counter.most_common()

[('2', 4), ('Q', 1)]

In [9]:
def secondary_compare(hand_one, hand_two):
    vals = ["A", "K", "Q", "T", "9", "8", "7", "6", "5", "4", "3", "2", "J"]
    strengths = {val: 15- i for i, val in enumerate(vals)}
    for i, char_one in enumerate(hand_one):
        char_b = hand_two[i]
        if char_b != char_one:
            if strengths[char_one] > strengths[char_b]:
                return 1
            return -1
    return 0
 

In [10]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 1200)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', -1)

/var/folders/pm/vw9pw9_93bl47zh9mtqphvb00000gn/T/ipykernel_58334/2812179107.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


In [11]:
df["strength"] = df.hand.apply(calculate_strength_joker)
strength_to_type = {i: hand_type for i, hand_type in enumerate(["high card", "one pair", "two pair", "three of a kind", "full house", "four of a kind", "five of a kind"])}
df["type"] = df.strength.replace(strength_to_type)
df

,hand,bid,strength,type
0,A2T63,467,0,high card
1,4854J,948,3,three of a kind
2,TJTT3,229,5,four of a kind
3,69664,839,3,three of a kind
4,ATT9A,340,2,two pair
...,...,...,...,...
995,Q7JQQ,350,5,four of a kind
996,5J597,883,3,three of a kind
997,QAQQK,936,3,three of a kind
998,42434,585,3,three of a kind


In [12]:
def compare(item1, item2):
    strength1 = calculate_strength_joker(item1)
    strength2 = calculate_strength_joker(item2)
    if strength1 != strength2:
        if strength1 > strength2:
            return 1
        return -1
    return secondary_compare(item1, item2)

In [13]:
compare("32T3K", "T55J5")

-1

In [14]:

compare("T55J5", "32T3K")

1

In [15]:
c

Counter({'Q': 3, 'J': 1, 'A': 1})

In [16]:

import functools
c = list(df.hand)
sortedl = sorted(c, key=functools.cmp_to_key(compare))

In [17]:
x = df.bid.copy()
x.index = df.hand
di = x.to_dict()

In [18]:
res = 0
ranks = {}
for rank, hand in enumerate(sortedl):
    #print(f"{rank+1} * {di[hand]} .... {hand}")
    res += (rank+1) * di[hand] 
    ranks[hand] = rank +1
res

249631254

In [19]:
pd.set_option('display.max_rows', None)
df["rank"] = df.hand.replace(ranks)
d1 = df.sort_values(by="rank")
d1

,hand,bid,strength,type,rank
195,25Q6K,110,0,high card,1
872,264A5,596,0,high card,2
121,265K7,850,0,high card,3
960,2684T,825,0,high card,4
244,2783Q,11,0,high card,5
164,27A64,457,0,high card,6
408,28546,515,0,high card,7
581,2976Q,949,0,high card,8
661,2T748,343,0,high card,9
460,2K69Q,539,0,high card,10
